Check what ids actually had related works to extract

In [ ]:
import json
import os
import shutil
ids = json.load(open('ids.json'))
related_ids = []
for id in ids:
    paper_path = f'papers/{id}/replacedcite.tex'
    if os.path.exists(paper_path):
        related_ids.append(id)
    else:
        dir_path = f'papers/{id}'
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)  # Delete the directory and all its contents
            print(f"Deleted directory: {dir_path}")

In [ ]:
print(len(related_ids))

Get .bib sections for papers with related works and add to file

In [ ]:
import requests
import tarfile
import time
for final_id in related_ids:
    print(f"Downloading... {final_id} at index {related_ids.index(final_id)} out of {len(related_ids)}")    

    url = f"https://arxiv.org/e-print/{final_id}"
    response = requests.get(url)

    # Create temp directory structure
    temp_dir = "temp/papers"
    os.makedirs(temp_dir, exist_ok=True)

    # Save the file
    filename = os.path.join(temp_dir, f"{final_id}")
    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f"Extracting files...")

    # Create output directory
    output_dir = f"papers/{final_id}"
    os.makedirs(output_dir, exist_ok=True)

    # Try to extract as tar.gz
    try:
        with tarfile.open(filename, "r:gz") as tar:
            members = tar.getmembers()
            
            # Filter for only .bib files
            bib_files = [member for member in members if member.name.endswith('.bib')]
            
            # Extract only the .tex files
            for bib_file in bib_files:
                tar.extract(bib_file, path=output_dir)
                print(f"Extracted: {bib_file.name}")
    except:
        print(f"Failed to extract: {final_id}, it may not be a tar.gz file: {response.headers['Content-Type']}")

    # Delete the temporary file
    os.remove(filename)
    print(f"Temporary file deleted: {filename}")
